In [9]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

In [20]:
csv_file_path = 'outbreaks.csv'
outbreaks_df = pd.read_csv(csv_file_path, sep='delimiter', header=None)
outbreaks_df.head()

D:\anaconda3\lib\site-packages\ipykernel_launcher.py:2: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  


,0
0,PK-    \tC4O ¬8(ÿÿÿÿÿÿÿÿ
1,  outbreaks.csv  x     AF     Ä½YsG²(üî...
2,ã
3,"�""\¶ßËÛº¬¢]^ç�y±©ÖÏ‡r%nŽö–ÏÜ…ø˜ÿÈ9{‚áª¸3Aa9Õw ..."
4,"Øw°í~Ÿ¯ �ûâp0¨Æè]¹s˜™Ž<ì«í¶0ÄÌÙ‚MýÀ[ª02WœS, r..."
